In [60]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os

from importlib import reload

In [98]:
# Load pre-processed ECG data matrix into memory

def load_ecg_data_matrix(filename='data_ecg_pickle'):
    '''
    Load ECG data matrix into memory
    Update global variable dict using keys read from pickle file
    '''
    import pickle

    with open('data_ecg.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        data_dict = pickle.load(f)

    return data_dict

data_dict =  load_ecg_data_matrix(filename='data_ecg_pickle')

for key in data_dict.keys():
    globals()[key] = data_dict[key]
    print('loaded ' + key + ' to memory')
    v = globals()[key]
    print( '  ' + str(type(v)) + '       ' + str(v.shape) )

del data_dict

loaded data_all to memory
  <class 'numpy.ndarray'>       (103504, 150)
loaded annotations_all to memory
  <class 'numpy.ndarray'>       (103504, 1)
loaded onsets_all to memory
  <class 'numpy.ndarray'>       (103504, 1)
loaded pt_codes_all to memory
  <class 'numpy.ndarray'>       (103504, 1)


In [99]:
# flag potential ecg artifacts: excessive variability and large negative deflections
# on individual ECG segments

data_all_shape_before_remove = data_all.shape

def remove_ecg_artifacts(data_all):
    data_all = data_all.copy()
    std = np.std(data_all, axis=1)
    std_th = std > np.mean(std) + 4*np.std(std)
    amp_max_neg = np.min(data_all, axis=1)
    amp_th = amp_max_neg < np.mean(amp_max_neg) - 4 * np.std(amp_max_neg)
    # amp_th.sum(), std_th.sum(), np.logical_and( amp_th , std_th).sum()

    ix = np.logical_and(~std_th, ~amp_th)
    data = data_all[ix,:]
    annotations = annotations_all[ix]
    onsets = onsets_all[ix]
    pt_codes = pt_codes_all[ix]

    n_removed = sum(np.logical_or(std_th, amp_th))
    n_kept = sum(np.logical_and(~std_th, ~amp_th))
    print( 'Flagged ' + str( n_removed ) + ' out of ' + str( len(std_th)) + ' ECG segments as artifacts ' )
    return data_all

In [45]:
# heart beat types
# N = Normal
# focus on A, L, R, V five most common heart beat anomaly types

tN = np.where(annotations=='N')[0]
tA = np.where(annotations=='A')[0]
tL = np.where(annotations=='L')[0]
tR = np.where(annotations=='R')[0]
tV = np.where(annotations=='V')[0]

#indices for normal ecg and five most common abnormal ecg types
ix = np.concatenate([tN, tA, tL, tR, tV])


data = data[ix,:]
annotations = annotations[ix]
onsets = onsets_all[ix]
pt_codes = pt_codes_all[ix]

labels_str = annotations
labels = np.zeros((len(labels_str), 1))
# for i, lab in enumerate(np.unique(labels_str)):
#     labels[labels_str==lab] = i
labels[labels_str=='N']=0
labels[labels_str=='A']=1
labels[labels_str=='L']=2
labels[labels_str=='R']=3
labels[labels_str=='V']=4


In [100]:
# Training  and test datasets:

# ECG data from 30 patients used for training (further split during cross-validation
# and in-sample testing (data from same patients used for both training and testing performance)
# ECG data from furhter 14 patients used for out-sample testing,
# to assess model performance on entirely new data

# pts_1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230]
# pts_2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]
pts_1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 121, 123, 200,
         201, 203, 205, 207, 208, 209, 215, 220, 223, 230, 228, 231, 232, 233, 234]
pts_2 = [100, 103, 105, 111, 113, 117, 202, 210, 212, 213, 214, 219, 221, 222]

ix_pts_1 = np.where( np.isin(pt_codes , pts_1))[0]
ix_pts_2 = np.where( np.isin(pt_codes , pts_2))[0]

data_pts_1, labels_pts_1, onsets_pts_1, pt_codes_pts_1 = \
    data[ix_pts_1,:], labels[ix_pts_1], onsets[ix_pts_1], pt_codes[ix_pts_1]

data_pts_2, labels_pts_2, onsets_pts_2, pt_codes_pts_2 = \
    data[ix_pts_2,:], labels[ix_pts_2], onsets[ix_pts_2], pt_codes[ix_pts_2]


ix_train = np.where( np.isin(pt_codes , pts_1))[0]
ix_test_out  = np.where( np.isin(pt_codes , pts_2))[0]

data_train, labels_train, onsets_train, pt_codes_train = \
    data[ix_train,:], labels[ix_train], onsets[ix_train], pt_codes[ix_train]

data_test_out, labels_test_out, onsets_test_out, pt_codes_test_out = \
    data[ix_test_out,:], labels[ix_test_out], onsets[ix_test_out], pt_codes[ix_test_out]


iX = list(range(data_train.shape[0]))
X = data_train
y = labels_train.ravel()

iX_train,  iX_test, y_train, y_test = \
    train_test_split(iX, y, test_size=0.25, stratify=y, random_state=0)

X_train, X_test = X[iX_train], X[iX_test]

X_test_out = data_test_out
y_test_out = labels_test_out



In [181]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

from sklearn.metrics import f1_score, precision_score, recall_score
from joblib import dump, load

In [200]:
# Baseline Model

# Random guess, assign class labels based on prior probability of each class
# Simulate 5-fold CV, compute f1, weighted average of f1 scores for each class

num_samples = X_train.shape[0]

ix = np.arange(0,num_samples)



n_20pc = num_samples // 5

f1_cv = np.zeros((5,))


for i in range(5):
    rnd = np.random.random(num_samples)
    ix_cv = ix[np.argsort(rnd)]
    ix_cv_train = ix_cv[n_20pc:]
    ix_cv_test = ix_cv[:n_20pc]
    y_cv_train = y_train[ix_cv_train]
    y_cv_test = y_train[ix_cv_test]
    p_vals = np.zeros((5,))
    for j in range(5):
        p_vals[j] = np.sum((y_cv_train==j)) / len(y_cv_train)
    p_vals = p_vals / np.sum(p_vals)
    # print(p_vals)
    y_cv_pred = np.argmax(np.random.multinomial(1, p_vals, len(y_cv_test)), axis=1)

    f1_cv[i] = f1_score(y_cv_test, y_cv_pred, average='weighted')


f1_baseline = np.round(np.mean(f1_cv) * 100,2)


print('Baseline - Random guess from multinomial distribution based on class probabilities')
print('expected f1 = ' + str(f1_baseline))
# expected f1 = 60.0

Baseline - Random guess from multinomial distribution based on class probabilities
expected f1 = 59.65


In [ ]:
# Model ECG abnormality with K Nearest Neighbors classifier
from sklearn.neighbors  import  KNeighborsClassifier
param_grid = { 'n_neighbors': [3,4,5,10,20]}
clf = KNeighborsClassifier()
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)
clf.fit(X_train, y_train.ravel())

y_test_pred = clf.predict(X_test)
y_out_pred = clf.predict(X_test_out)

In [108]:
clf_kNN = clf
dump(clf_kNN, 'Model_kNN'  + '.joblib')

['Model_kNN.joblib']

In [111]:
print('Best performing K Nearest Neighbors model\n')
print(clf.best_estimator_)

print(' ')

print('kNN classification performance - Train-test on same group of individuals\n')
print(classification_report(y_test, y_test_pred,     target_names=['N','A','L','R','V']))

print(' ')

print('kNN classification performance - Train on group 1, test on group 2\n')
print(classification_report(y_test_out, y_out_pred,     target_names=['N','A','L','R','V']))




Best performing K Nearest Neighbors model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
 
kNN classification performance - Train-test on same group of individuals

              precision    recall  f1-score   support

           N       0.99      1.00      1.00     12492
           A       0.98      0.92      0.95       524
           L       0.99      0.99      0.99       672
           R       0.99      1.00      0.99      1338
           V       0.98      0.97      0.98      1373

    accuracy                           0.99     16399
   macro avg       0.99      0.97      0.98     16399
weighted avg       0.99      0.99      0.99     16399

 
kNN classification performance - Train on group 1, test on group 2

              precision    recall  f1-score   support

           N       0.85      0.98      0.91     24409
           A       0.19   

In [115]:
from sklearn.linear_model import LogisticRegression

#clf = LogisticRegression( C=100, penalty='l2', solver='saga', tol=0.01, class_weight='balanced')

param_grid = { 'C': [ 0.01, 0.1, 1, 10, 100, ]}
clf = LogisticRegression(class_weight='balanced', solver='sag', max_iter=5000)
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train.ravel())

y_test_pred = clf.predict(X_test)
y_out_pred = clf.predict(X_test_out)

clf_LogReg = clf
dump(clf_LogReg, 'Model_LogReg'  + '.joblib')

print('Best performing Logistic Regression model\n')
print(clf.best_estimator_)

print(' ')

print('LogReg classification performance - Train-test on same group of individuals\n')
print(classification_report(y_test, y_test_pred,     target_names=['N','A','L','R','V']))

print(' ')

print('LogReg classification performance - Train on group 1, test on group 2\n')
print(classification_report(y_test_out, y_out_pred,     target_names=['N','A','L','R','V']))


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best performing Logistic Regression model

LogisticRegression(C=100, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=5000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
 
LogReg classification performance - Train-test on same group of individuals

              precision    recall  f1-score   support

           N       0.99      0.88      0.93     12492
           A       0.50      0.88      0.63       524
           L       0.82      0.97      0.89       672
           R       0.83      0.96      0.89      1338
           V       0.63      0.91      0.74      1373

    accuracy                           0.89     16399
   macro avg       0.75      0.92      0.82     16399
weighted avg       0.92      0.89      0.90     16399

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed: 15.2min remaining: 11.9min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 21.5min finished


In [116]:
from imblearn.ensemble import BalancedRandomForestClassifier

param_grid = { 'n_estimators': [50, 100, 1000],
               'max_features': [ 'sqrt', 'log2', None],
                'class_weight': ['balanced', 'balanced_subsample']}
clf = BalancedRandomForestClassifier(oob_score = True)
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train.ravel())


y_test_pred = clf.predict(X_test)
y_out_pred = clf.predict(X_test_out)

clf_bRF = clf
dump(clf_bRF, 'Model_LogReg'  + '.joblib')

print('Best performing Balanced Random Forest model\n')
print(clf.best_estimator_)

print(' ')

print('bRF classification performance - Train-test on same group of individuals\n')
print(classification_report(y_test, y_test_pred,     target_names=['N','A','L','R','V']))

print(' ')

print('bRF classification performance - Train on group 1, test on group 2\n')
print(classification_report(y_test_out, y_out_pred,     target_names=['N','A','L','R','V']))

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best performing Balanced Random Forest model

BalancedRandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                               class_weight='balanced_subsample',
                               criterion='gini', max_depth=None,
                               max_features='log2', max_leaf_nodes=None,
                               max_samples=None, min_impurity_decrease=0.0,
                               min_samples_leaf=2, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=1000,
                               n_jobs=None, oob_score=True, random_state=None,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)
 
bRF classification performance - Train-test on same group of individuals

              precision    recall  f1-score   support

           N       1.00      0.97      0.99     12492

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
/home/at/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 15.3min finished


In [ ]:
# pts_100 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 121, 123,
#            100, 103, 105, 111, 113, 117]
# pts_200 = [200, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230, 228, 231, 232, 233, 234]
#
# pts_100